## Basic Algorithms: Fibonacci

Another form of recursion is tree recursion. Consider computing a fibonacci sequence, in which each number is the sum of the previous two, with the first two taken to be 0 and 1.

### Recursive Fibonacci. 


We write Fibonacci recursively with the first two numbers as base cases.

![](https://mitpress.mit.edu/sicp/full-text/book/ch1-Z-G-13.gif)

(from SICP)

Signature: `def fib_recursive(n)`

In [29]:
# recursion
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

# here is a 1-index 
for i in [1, 2, 7, 13, 29, 33]:
    print(fib_recursive(i))

0
1
8
144
317811
2178309


### Q1. 

What are the space and time complexities of this implementation? Hint: think aboutthe number of items in a binary tree and its depth.

**Answer**

**Time complexity**<br>

Since ultimately ``fib_recursive(n)`` has to reduce to the leaf nodes of either ``fib_recursive(0)`` or ``fib_recursive(1)``, effectively, this means that n is the depth of the tree. Therefore the number of calls to this function is 2^n, hence the time complexity is $O(2^n)$

**Space complexity**<br>

For space, the consumption of memory is that of stack frame in the longest chain of recursive calls that are pending. That will be the depth of the tree, which we have determined above to be n for ``fib_recursive(n)``. At any given time, only one branch is recursively pending execution. Therefore the space complexity is $O(n)$, where n reflects the size of the longest branch.


### Dynamic Programming and Iteration

From Skiena
>..**dynamic programming**, which typically removes one element from the problem, solves the smaller problem, and then uses the solution to this smaller problem to add back the element in the proper way. **Divide-and-conquer** instead splits the problem in (say) halves, solves each half, then stitches the pieces back together to form a full solution.

>Dynamic programming is a technique for efficiently implementing a recursive algorithm by storing partial results. The trick is seeing whether the naive recursive algorithm computes the same subproblems over and over and over again. If so, storing the answer for each subproblems in a table to look up instead of recompute can lead to an efficient algorithm. Start with a recursive algorithm or definition. Only once we have a correct recursive algorithm do we worry about speeding it up by using a results matrix. Dynamic programming is generally the right method for optimization problems on combinatorial objects that have an inherent left to right order among components. Left-to-right objects includes: character strings, rooted trees, polygons, and integer sequences.

### Q2.

Here is an implementation of Fibonacci using dynamic programming: they key is to notice that the recurrence we used can be put into an iterative form and just stored in an ever increasing array. What is the space and time complexity here? What if you were somehow able to save the array outside of the function when u calculate `fib(M)` and subsequently had to calculate `fib(N)`, where $N>M$.

In [30]:
def fib_iterative(n):
    fibs=[]
    fibs.append(0)
    fibs.append(1)
    for i in range(2, n):
         fibs.append(fibs[i-1]+fibs[i-2])
    return fibs[n-1]

In [31]:
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative(i))

0
1
8
144
317811
2178309


**Part 1**

*What is the space and time complexity here?*

**Time Complexity**

There is a fixed cost of $O(1)$ for all the other lines except the 'for loop' as they are executed exactly once. Counting the loop operations, with k other lines, we have $k*(n-2)$, which the time complexity can be generalized to $O(n)$.

**Space Complexity**

Since the for loop creates a list of size n with the append function, the space complexity is $O(n)$

**Part 2**
<br>

*What if you were somehow able to save the array outside of the function when u calculate fib(M) and subsequently had to calculate fib(N), where  $N>M$ *
<br>

**Space Complexity**

As `fib(n)` still has to store the array of (n-m), hence the space complexity is $O(n)$.

**Time Complexity**

For $n>m$ up to fib(n), we have to find the time complexity of fib(m+1), fib(m+2),..., fib(n). $O(m-n)$ becomes asymptotic to $O(n)$. The fetching of the values from fib(m) is a constant, k. Hence the final time complexity is $O(n+k)$, which is $O(n)$




### Recursive Fibonacci with cacheing

### Q3. 

Use `cache` and `call_counter` as decorators on `fib_recursive` and print the fibonacci numbers for 7,13,29, 33. What order should these decorators be called to make sure `call_counter` gets the actual number of calls to `fib_recursive`?

We've written the `cache` decorator for you. You have to write the `call_counter` decorator which takes the function as argument and using a `count_dictionary` whose keys are function names, counts the number of times the function is called.

In [9]:
def cache(f):
    """f is a single argument function whose values may be cached"""
    # creating an empty dictionary of cache
    cache = {}
    def memoized_func(x):
        #pruning the call tree by the if function
        if x not in cache:
            cache[x] = f(x)
        return cache[x]
    memoized_func.__name__ = f.__name__
    #helps create a proxy function and call it fib recursive
    return memoized_func
    

In [8]:
def call_counter(count_dictionary):
    def call_counter_decorator(func):
        name = func.__name__
        count_dictionary[name] = 0
        def inner(*args, **kwargs):
            count_dictionary[name] = count_dictionary[name] + 1
            return func(*args, **kwargs)
        return inner
    return call_counter_decorator

In [11]:
ccounter={}
@cache
@call_counter(ccounter)   # f = call_counter(f,ccounter)
#@cache
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

for i in [ 7,13,29, 33]:
    print(i, fib_recursive(i), ccounter['fib_recursive'])

7 8 7
13 144 13
29 317811 29
33 2178309 33


**Solution**

**Order of the decorators** <br>
`@cache` <br>
`@call_counter (ccounter)`<br>

**Reasons** <br>
We must decorate with call_counter first to count the actual number of calls to fib_recursive, otherwise, we will be counting the calls to cache decorated fib_recursive.



### Q4.

What is the time and space complexity of the the memoized Fibonacci? HINT: assume evaluation happens left to right on sub-expressions, so that `fib_recursive(n-1)` side of the tree is evaluated first, and thus the tree is evaluated depth first, from left to right. What kind of pruning happens in the tree?

**Time Complexity**

For `fib(n)`, the depth of the tree will be n. As we have pruned the tree with the cache decorator, we effectively need to go down the left most path of the tree. The memo-ization will effectively prune all the right children. Hence the time complexity is $O(n).

**Space Complexity**

This will stay the same as the naive algorithm above in Question 1, since the consumption of memory is still that of stack frame in the longest chain of recursive calls that are pending. Hence the space complexity here for ``fib_recursive(n)`` with decorators is $O(n)$.

**Pruning** 

The memo-ization will effectively prune all the right hand branches of the tree, except for the left-most fib(1) leaf, as we start with fib(1), fib(2) as the base cases.


### Q5.

Do you really need to store the entire array in the dynamic programming implementation? Isnt it enough to have only saved the previous two Fibonacci numbers? Implement such an algorithm in `fib_iterative2(n)`. What is its space and time complexity? How is the time complexity different from that of the previous iterative attempt?

In [1]:
def fib_iterative2(n):
    if (n==1):
        return 0
    a = 0
    b = 1
    for i in range(3, n+1):
        c = a + b
        a = b
        b = c
    return b

In [2]:
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative2(i))

0
1
8
144
317811
2178309


The above algorithm's time complexity is $O(n)$. The time complexity remains the same from the previous iterative attempt. However, the space complexity improves to $O(1)$ since the memory usage is constant.

### Q6.

Write an algorithm for insertion sort.

![](https://camo.githubusercontent.com/8f6fedc10da579f13b22b949f6ad29255b6d721f/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f302f30662f496e73657274696f6e2d736f72742d6578616d706c652d33303070782e676966)

(from wikipedia)

The algorithm is also illustrated here: http://cs.armstrong.edu/liang/animation/web/InsertionSort.html and may be described thus:

Insertion sort is a method for sorting that starts with a single element (thus forming a trivially sorted list) and then incrementally inserts the remaining elements so that the list stays sorted.

Talk about the best, worst and average complexity of insertion sort. Use the A=[5,2,1,3,8,6,9] to show your code!

In [5]:
def insertion_sort(array):
    for slot in range(1, len(array)): 
        # iterate from second element of the array onwards
        value = array[slot] #value is the second element onwards
        test_slot = slot - 1 
        # if the left hand side is greater then shift the LHS to the right
        while test_slot >= 0 and array[test_slot] > value:
            array[test_slot + 1] = array[test_slot]
            #shift further to the left
            test_slot = test_slot - 1
        # assign the value captu red back
        array[test_slot + 1] = value
    return array

In [6]:
A=[5,2,1,3,8,6,9]

insertion_sort(A)

[1, 2, 3, 5, 6, 8, 9]

**Best Case Time Complexity** <br>

The insertion sort scans through the list of n elements, compare one element with another element on the left (in pairs), then it swaps the elements if they are out of order. Hence there will be at least n operations (length of the array) that sum up to the total time complexity of the algorithm. In the best case scenario, the input array is already sorted, hence the algorithm merely compares the elements as it runs down the array and performs no swaps. This means that in the code above, the while loop is never triggered. As such the time complexity is $O(n)$.

**Worst and Average Case Time Complexity** <br>

Assuming that we would like to sort the array in ascending order, the worst case scenario will occur when the input array is in descending order. To insert the last element, we need a maximum of $n-1$ comparisons and $n-1$ swaps; similary, to insert the second last element, there will be $n-2$ comparisons and $n-2$ swaps. Hence the total number of operations will be:

$$2(1+2+...+ n-2+ n-1)$$

Following the summation notation:
$$\sum_{q=1}^{p} q = \frac{p(p+1)}n $$

The time complexity is:
$$\frac{2(n-1)(n-1+1)}2 = n(n-1)$$

When simplified, the time complexity is $O(n^2)$

**Space Complexity**

Assuming the array is passed as a parameter, the space complexity is is $O(1)$ because the memory allocation is constant and allocated for. If on the other hand, the array space consumption is charged to the insertion sort function, then the space complexity is $O(n)$.